In [ ]:
#
#
# *** Image: Base Python 3.0
# *** Kernel: Python 3
# *** Instance type: ml.m5.large
# *** Start-up script: No script 
#
#
# This example requires two files in the environment:
#
#   1. new_dist/amazon_fmeval-*-py3-none-any.whl
#   2. tiny_dataset.jsonl
#
# This example will write its output to:
#
#   /tmp/eval_results/factual_knowledge_tiny_dataset.jsonl
#
#

!pip3 install sagemaker

!pip3 install -U pyarrow
!pip3 install -U accelerate
!pip3 install "ipywidgets>=8"

In [ ]:
import sagemaker

In [ ]:
#
# Let's check for this example's required files in the environment:
#
#   1. new_dist/amazon_fmeval-*-py3-none-any.whl
#   2. tiny_dataset.jsonl
#

import glob

if not glob.glob("new_dist/amazon_fmeval-*-py3-none-any.whl"):
    print("ERROR - please make sure file exists: new_dist/amazon_fmeval-*-py3-none-any.whl")

if not glob.glob("trex_sample.jsonl"):
    print("ERROR - please make sure file exists: trex_sample.jsonl")

In [ ]:
#
# Install the amazon_fmeval-*-py3-none-any.whl distribution.
#

!rm -Rf ~/.cache/pip/*

!pip3 install new_dist/*.whl --upgrade --upgrade-strategy only-if-needed --force-reinstall

In [ ]:
import warnings
from dataclasses import dataclass
from typing import Tuple, Optional

import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

from amazon_fmeval.model_runners.model_runner import ModelRunner
from amazon_fmeval import get_eval_algorithm
from amazon_fmeval.eval_algorithms.factual_knowledge import FactualKnowledgeConfig

In [ ]:
@dataclass
class HFModelConfig:
    model_name: str
    max_new_tokens: int
    normalize_probabilities: bool = False
    seed: int = 0
    remove_prompt_from_generated_text: bool = True


class HuggingFaceCausalLLMModelRunner(ModelRunner):
    def __init__(self, model_config: HFModelConfig):
        self.config = model_config
        self.model = AutoModelForCausalLM.from_pretrained(self.config.model_name)
        self.tokenizer = AutoTokenizer.from_pretrained(self.config.model_name)

    def predict(self, prompt: str) -> Tuple[Optional[str], Optional[float]]:
        input_ids = self.tokenizer(prompt, return_tensors="pt").to(self.model.device)
        generations = self.model.generate(
            **input_ids,
            max_new_tokens=self.config.max_new_tokens,
            pad_token_id=self.tokenizer.eos_token_id,
        )
        generation_contains_input = (
            input_ids["input_ids"][0] == generations[0][: input_ids["input_ids"].shape[1]]
        ).all()
        if self.config.remove_prompt_from_generated_text and not generation_contains_input:
            warnings.warn(
                "Your model does not return the prompt as part of its generations. "
                "`remove_prompt_from_generated_text` does nothing."
            )
        if self.config.remove_prompt_from_generated_text and generation_contains_input:
            output = self.tokenizer.batch_decode(generations[:, input_ids["input_ids"].shape[1] :])[0]
        else:
            output = self.tokenizer.batch_decode(generations, skip_special_tokens=True)[0]

        with torch.inference_mode():
            input_ids = self.tokenizer(self.tokenizer.bos_token + prompt, return_tensors="pt")["input_ids"]
            model_output = self.model(input_ids, labels=input_ids)
            probability = -model_output[0].item()

        return output, probability

# Test with gpt2
hf_config = HFModelConfig(model_name="gpt2", max_new_tokens=32)
model = HuggingFaceCausalLLMModelRunner(model_config=hf_config)
print(model.predict("London is the capital of?"))

In [ ]:
# Evaluate factual_knowledge
eval_algorithm_config = FactualKnowledgeConfig("<OR>")
eval_algo = get_eval_algorithm("factual_knowledge")(eval_algorithm_config)

In [ ]:
# Evaluate your custom sample
model_output = model.predict("London is the capital of?")[0]
print(model_output)
eval_algo.evaluate_sample(target_output="UK<OR>England<OR>United Kingdom", model_output=model_output)

In [ ]:
# Custom dataset
from amazon_fmeval.data_loaders.data_config import DataConfig
dataset_config = DataConfig(
        dataset_name="TREX",
        dataset_uri="trex_sample.jsonl",
        dataset_mime_type="application/jsonlines",
        model_input_location="question",
        target_output_location="answers",
        category_location="knowledge_category",
    )

# Evaluate model with amazon-fmeval built-in dataset
eval_outputs = eval_algo.evaluate(model=model, dataset_config=dataset_config, prompt_template="$feature", save=True)

In [ ]:
# Show Evaluation outputs
print(eval_outputs)

In [ ]:
# Show first five rows of saved output
with open('/tmp/eval_results/factual_knowledge_TREX.jsonl') as f:
	lines = [next(f) for _ in range(5)]
for line in lines:
	print(line)